In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import os

In [7]:
def count_missing(fullfilename):
    df = pd.read_csv(fullfilename)
    print(df.isna().sum())

In [ ]:
def create_map(fullfilename):
    df = pd.read_csv(fullfilename)
    

In [71]:
filepath = "ncbi\\aka_scraped"

In [19]:
def get_dups(fullfilepath):    
    df = pd.read_csv(fullfilepath)

    cols = ['LncRNA name', 'Cancer name', 'Methods', 'Expression pattern',
        'Pubmed ID', 'aka']

    present_map = {}
    dup_indices = []
    for index, row in df[cols][:-2].iterrows():
        curentry = tuple(row[['LncRNA name', 'Cancer name', 'Methods', 'Expression pattern',
        'Pubmed ID',]])
        present_map[curentry] = (False, [])

    for index, row in df[cols][:-2].iterrows():
        curentry = tuple(row[['LncRNA name', 'Cancer name', 'Methods', 'Expression pattern',
        'Pubmed ID',]])
        
        if present_map[curentry][0] == True:
            # dup_indices.append((index, present_map[curentry][1]))
            present_map[curentry][1].append(index)
        else:
            akas = str(list(row[['aka']])[0]).strip().split(';')

            for aka in akas:
                tmplist = list(curentry)
                tmplist[0] = aka
                akaentry = tuple(tmplist)

                if akaentry in present_map.keys() and present_map[akaentry][0] == True:
                    # dup_indices.append((index, present_map[curentry][1]))
                    present_map[akaentry][1].append(index)

            present_map[curentry] = (True, [index])

    return {fullfilepath: present_map}

In [68]:
delete_rows = {}

for filename in os.listdir(filepath):
    if(filename[-3:]=='csv'):
        fullfilename = os.path.join(filepath, filename)
        dups_data = get_dups(fullfilename)

        removerows = []

        for i in dups_data.values():
            for j in i:
                if len(i[j][1]) > 1:
                    for rownum in i[j][1][1:]:
                        removerows.append(rownum)

        delete_rows[filename] = removerows

import json
with open('final_dups_to_remove.json', 'w') as fp:
    json.dump(delete_rows, fp)

# remove row number+2 wali row in excel from the sheet

In [76]:
for filename in os.listdir(filepath):
    if(filename[-3:]=='csv'):
        fullfilename = os.path.join(filepath, filename)
        df = pd.read_csv(fullfilename)
        rows_to_delete = delete_rows[filename]
        if len(rows_to_delete)>0:
            df.drop(df.index[rows_to_delete], inplace=True)
            df.to_csv(os.path.join('ncbi\\aka_scraped\\dups_removed', 'dups_removed_'+filename), index=False)

In [31]:
combined_df = pd.read_csv('lnccancer_db\\combined.csv', header=None, index_col=0)
aka_df = pd.read_csv('aka_db.csv', header=None, index_col=0)
aka_df.columns = ['LncRNA name', 'aka']

# print(combined_df.columns)
combined_df = combined_df[combined_df.columns[:-1]]
# print(combined_df.head())
combined_df.columns = ['LncRNA name', 'Cancer name', 'Methods', 'Expression pattern',
        'Pubmed ID', 'No. of Transcript variants']

# print(combined_df.head())

In [38]:
grouped_aka = aka_df.groupby('LncRNA name')
list(grouped_aka.get_group('AATBC')['aka'])

['apoptosis associated transcript in bladder cancer',
 'apoptosis-associated transcript in bladder cancer',
 'AATBC',
 'HSALNG0133454',
 'NONHSAG032997.2',
 'HSALNG0133458']

In [45]:
combined_df.head()

,LncRNA name,Cancer name,Methods,Expression pattern,Pubmed ID,No. of Transcript variants
0,,,,,,
0,ABHD11-AS1,Colorectal Cancer,"qPCR, RIP, Luciferase reporter assay, Western ...",up-regulated,30429229.0,1.0
1,ABHD11-AS1,Colorectal Cancer,"qPCR, Western blot, Luciferase reporter assay,...",up-regulated,30537177.0,1.0
2,ADAMTS9-AS2,Colorectal Cancer,qPCR etc.,down-regulated,27596298.0,1.0
3,ADPGK-AS1,Colorectal Cancer,"qPCR, Luciferase reporter assay etc",up-regulated,32196589.0,1.0
4,AFAP1-AS1,Colorectal Cancer,"qPCR, Western blot, in vitro knockdown, RIP",up-regulated,29888110.0,1.0


In [48]:
present_map = {}
err_cnt = 0
for index, row in combined_df.iterrows():
    curentry = tuple(row[combined_df.columns])[:6]
    if(len(curentry)!=6):
        print(index, row)
    present_map[curentry] = (False, [])

for index, row in combined_df.iterrows():
    curentry = tuple(row[combined_df.columns])[:6]


    
    if present_map[curentry][0] == True:
        # dup_indices.append((index, present_map[curentry][1]))
        present_map[curentry][1].append(index)
    else:
        try:

            akas = list(grouped_aka.get_group(row['LncRNA name'])['aka'])

            for aka in akas:
                tmplist = list(curentry)
                tmplist[0] = aka
                akaentry = tuple(tmplist)

                if akaentry in present_map.keys() and present_map[akaentry][0] == True:
                    # dup_indices.append((index, present_map[curentry][1]))
                    present_map[akaentry][1].append(index)
        except:
            err_cnt+=1

        present_map[curentry] = (True, [index])

print(err_cnt)

KeyError: ('AATBC', 'Nasopharyngeal Cancer', 'Western blot, qRT-PCR, Luciferase reporter assay etc.', 'up-regulated', 32364663.0, nan)